In [2]:
# import json 
# with open("/mnt/xingjian_luo/dataset/llava_plus_dataset/json_dataset_modified/tool_origin_20000.json","r") as f:
#     dataset = json.load(f)

# with open("/mnt/xingjian_luo/dataset/datasets--LLaVA-VL--llava-plus-data/tool_70k_coco.json","r") as f:
#     dataset = json.load(f)
# use_tool_thoughts_list = []
# for i in dataset:
#     if len(i["conversations"][1]["actions"])>0:
#         use_tool_thoughts = i["conversations"][1]["thoughts"]
#         use_tool_thoughts_list.append(use_tool_thoughts)

# with open("use_tools_thought_list.json","w") as f:
#     f.write(json.dumps(use_tool_thoughts_list,indent=2))
# len(use_tool_thoughts_list)


In [33]:
import os
import json
import numpy as np


In [43]:
files = os.listdir("/mnt/xingjian_luo/dataset/CholecT50/videos")
np.random.seed(42)
np.random.shuffle(files)
train_len = int(0.8*len(files))

train_split = files[0:train_len]
test_split = files[train_len:]

In [44]:
test_split

['VID23',
 'VID56',
 'VID12',
 'VID13',
 'VID52',
 'VID25',
 'VID103',
 'VID66',
 'VID35',
 'VID74']

In [20]:
with open("/mnt/xingjian_luo/dataset/llava_plus_dataset/json_dataset_modified/detect_segment_01.json","r") as f:
    anno_1 = json.load(f)

with open("/mnt/xingjian_luo/dataset/llava_plus_dataset/json_dataset_modified/detect_segment_02.json","r") as f:
    anno_2 = json.load(f)

In [21]:
ano_seg = []
ano_det = []

for i in anno_1:
    if i["conversations"][1]["actions"][0]["API_name"] == "detect_model":
        ano_det.append(i)
    else:
        ano_seg.append(i)

for i in anno_2:
    if i["conversations"][1]["actions"][0]["API_name"] == "detect_model":
        ano_det.append(i)
    else:
        ano_seg.append(i)

In [28]:
with open("/mnt/xingjian_luo/dataset/surgFC/dataset/ano_seg/ano_seg.json","w") as f:
    f.write(json.dumps(ano_seg,indent=2))
with open("/mnt/xingjian_luo/dataset/surgFC/dataset/ano_det/ano_det.json","w") as f:
    f.write(json.dumps(ano_det,indent=2))    

In [9]:
import os

len(os.listdir("/mnt/xingjian_luo/dataset/coco_xiehe_segment/labels/val"))

126

In [5]:
import os
import json
import time
from tqdm import tqdm
import fire

from openai import OpenAI

import concurrent.futures
import random
import json
import time
from collections import Counter
from functools import partial

from pycocotools.coco import COCO

import requests
from PIL import Image
import base64
import json
import time
from io import BytesIO

import torchvision.transforms.functional as F

client = OpenAI(api_key="sk-sqilImjoPeMWLosGC7EbB5Dc215d4320BbDa49C59f73Eb85", 
                base_url="https://vip.yi-zhan.top/v1")

In [7]:
def ask_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt},
        ],
        stream=False,
        temperature=1.25
    )
    return response.choices[0].message.content

In [ ]:
def generate_worker(captions_strs, objects_strs, examples, sample, image_dir):
    # 1. captions_strs + objects_strs -> questions
    # 2. questions -> grounding dino input
    # 3. grounding dino input -> grounding dino output
    # 4. captions_strs + objects_strs + questions + grounding dino output -> answer

    # 1. captions_strs + objects_strs -> questions
    messages = [
        {'role': 'system', 'content': """You are an AI visual assistant that can analyze a single image. You receive five sentences, each describing the same image you are observing. In addition, specific object locations within the image are given, along with detailed coordinates. These coordinates are in the form of bounding boxes, represented as (x1, y1, x2, y2) with floating numbers ranging from 0 to 1. These values correspond to the top left x, top left y, bottom right x, and bottom right y.
Generate a question that users may be interested to ask about the image. The question should ask the AI to detect some objects in the image. The question should be answerable by the given sentences and the given object locations.
The question should ask the AI to detect some objects in the image."""},
    {"role": "user", "content": examples[0]['captions']+'\n'+examples[0]['objects']},
    {"role": "assistant", "content": examples[0]['question']},
    {"role": "user", "content": examples[1]['captions']+'\n'+examples[1]['objects']},
    {"role": "assistant", "content": examples[1]['question']},
    {"role": "user", "content": captions_strs + '\n' + objects_strs}
    ]
    question = ask_gpt(messages, temperature=0.9, top_p=0.95)
    if question is None:
        print("question is None, return None")
        return None

    # 2. questions -> grounding dino input
    messages = [
        {'role': 'system', 'content': """You are an AI visual assistant that can help to extract information from an a sentence. 
You will be given a question about detecting something in an image. Please extract the main object name from the question. Using '.' to concat multiple object names."""},
    {"role": "user", "content": examples[0]['question']},
    {"role": "assistant", "content": examples[0]['grounding_dino_input']},
    {"role": "user", "content": examples[1]['question']},
    {"role": "assistant", "content": examples[1]['grounding_dino_input']},
    {"role": "user", "content": "Please detect the green car in the image."},
    {"role": "assistant", "content": "green car"},
    {"role": "user", "content": question}
    ]
    grounding_dino_input = ask_gpt(messages, temperature=0.9, top_p=0.95)
    if grounding_dino_input is None:
        print("grounding_dino_input is None, return None")
        return None

    # 3. grounding dino input -> grounding dino output
    # get grounding dino output
    
    worker_addr = get_worker_addr(controller_address, model_name)
    headers = {"User-Agent": "GSAM Client"}
    # img_path = os.path.join(args.image_dir, image['image_id'])
    img_path = os.path.join(image_dir, sample['file_name'])
    img = load_image(img_path)
    img_arg = encode(img)
    ret = requests.post(
            worker_addr + "/worker_generate",
            json={
                "image": img_arg,
                "caption": grounding_dino_input,
                "box_threshold": 0.3,
                "text_threshold": 0.25,
            },
            headers=headers,
        ).json()
    if os.getenv('DEBUG_PRINT'):
        print(ret)
    ret.pop("size")
    grounding_dino_output = ret

    # 4. captions_strs + objects_strs + questions + grounding dino output -> answer
    q_temp = "caption: {cap}\ngrounding dino input: {gdin}\ngrounding dino output: {gdout}\nquestion: {q}\n"
    messages = [
        {'role': 'system', 'content': """You are an AI visual assistant that can analyze a single image. 
You receive five sentences, each describing the same image you are observing. 
Then you receive the output of the grounding dino model, with its corresponding input of grounding dino. The output is a list of objects detected in the image, with their corresponding bounding boxes. These coordinates are in the form of bounding boxes, represented as (x1, y1, x2, y2) with floating numbers ranging from 0 to 1. These values correspond to the top left x, top left y, bottom right x, and bottom right y.
Then you receive the question asked by the user. 
Answer the question based on the given information with your best. 
Do not reveal the input information of the image. DO NOT say that you are given the captions and the objects in the image, JUST answer the question as if you are seeing the image for the first time."""},
        {'role': 'user', 'content': q_temp.format(cap=examples[0]['captions'], gdin=examples[0]['grounding_dino_input'], gdout=examples[0]['grounding_dino_output'], q=examples[0]['question'])},
        {'role': 'assistant', 'content': examples[0]['answer']},
        {'role': 'user', 'content': q_temp.format(cap=examples[1]['captions'], gdin=examples[1]['grounding_dino_input'], gdout=examples[1]['grounding_dino_output'], q=examples[1]['question'])},
        {'role': 'assistant', 'content': examples[1]['answer']},
        {'role': 'user', 'content': q_temp.format(cap=captions_strs, gdin=grounding_dino_input, gdout=grounding_dino_output, q=question)},
    ]
    answer = ask_gpt(messages, temperature=0.9, top_p=0.95)
    if answer is None:
        print("answer is None, return None")
        return None

    # return
    return {
        "unique_id": str(time.time()) + '_' + str(sample['id']),
        "image_id": sample['id'],
        "image_file_name": sample['file_name'],
        "image_path": os.path.join(image_dir, sample['file_name']),
        "captions": captions_strs,
        "objects": objects_strs,
        "question": question,
        "grounding_dino_input": grounding_dino_input,
        "grounding_dino_output": grounding_dino_output,
        "answer": answer,
    }

